In [1]:
import os

os.chdir("../../")
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
model = AutoModelForSequenceClassification.from_pretrained(
    "abhishek/autonlp-bbc-news-classification-37229289")

tokenizer = AutoTokenizer.from_pretrained(
    "abhishek/autonlp-bbc-news-classification-37229289")

In [3]:
target_dir = os.getcwd() + "/data/text/solomon_islands/"
st = pd.read_csv(target_dir + "solomon_times_news.csv").drop("Unnamed: 0", axis=1)
st["tag"] = st["tag"].apply(lambda x: x.split(",")[-1].lower())

In [4]:
predicted_labels = []
with tqdm(total=len(st)) as pbar:
    for idx, news in enumerate(st.news):
        inputs = tokenizer(str(news), truncation=True, return_tensors="pt")
        output = model(**inputs)
        probs = F.softmax(output["logits"], dim=1)
        predicted_labels.append(int(torch.argmax(probs)))
        pbar.update(1)

  0%|▏                                                                  | 30/11049 [00:08<50:51,  3.61it/s]


KeyboardInterrupt: 

In [5]:
st["predicted_label"] = predicted_labels
st["predicted_label"] = st["predicted_label"].map({
    0: "economy",
    1: "entertainment",
    2: "politics",
    3: "sports",
    4: "tech"
})
st

,url,date,news,tag,predicted_label
0,https://www.solomontimes.com/news/north-vella-...,2007-04-28 20:25:26,The psychological effect of the tsunami is for...,economy,politics
1,https://www.solomontimes.com/news/british-high...,2007-04-30 02:00:40,The British High Commission Office in Honiara ...,politics,economy
2,https://www.solomontimes.com/news/internationa...,2007-04-30 21:01:39,"Afu Billy, the regional director of the common...",politics,economy
3,https://www.solomontimes.com/news/ratification...,2007-04-30 20:37:59,"Ken Averre, Director of the Public Prosecution...",politics,politics
4,https://www.solomontimes.com/news/un-human-rig...,2007-04-30 12:29:00,A public forum was today organized by the Offi...,politics,economy
...,...,...,...,...,...
11044,https://www.solomontimes.com/news/nha-clarifie...,2023-08-07 00:53:00,The National Hosting Authority (NHA) has been ...,sports,sports
11045,https://www.solomontimes.com/news/big-fish-beh...,2023-08-07 22:18:00,Member of Parliament for West Kwaio Constituen...,politics,politics
11046,https://www.solomontimes.com/news/pacific-reco...,2023-08-07 23:58:00,Despite uncertainty about the outlook for infl...,economy,economy
11047,https://www.solomontimes.com/news/sogavare-for...,2023-08-07 00:45:00,Prime Minister Manasseh Sogavare has declared ...,politics,economy


In [8]:
len(st[(st.tag == st.predicted_label) & (st.tag == "economy")])/len(st)

0.41677979907683954